In [1]:
print("ok")

ok


In [2]:
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain


from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [3]:
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf",
                     loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents

In [4]:
!pip install pypdf
extracted_data = load_pdf("data/")

  Using cached pypdf-5.7.0-py3-none-any.whl.metadata (7.2 kB)
Using cached pypdf-5.7.0-py3-none-any.whl (305 kB)


Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEnc

In [ ]:
#extracted_data

In [5]:
#Create text Chunks now

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [6]:
text_chunks = text_split(extracted_data)

In [7]:
text_chunks = text_split(extracted_data)
# to determine the length of text chunks
print("lengh of text chunks: ", len(text_chunks))

lengh of text chunks:  3433


In [8]:
# text_chunks

In [9]:
# Download Embedding Model

def download_huggingface_embedding_model():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings
                                         

In [ ]:
!pip install sentence-transformers
embeddings= download_huggingface_embedding_model()

In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
query_result = embeddings.embed_query("What is the treatment for diabetes?")
print("Length", len(query_result))

Length 384


In [62]:

import os

os.environ["PINECONE_API_KEY"] = "pcsk_3gLL6_S9SetKF1d8DEnzXA6TdACPwjx5CuntHRY3BDH9vbJ652vVhtfXBq6RpgUfz1ijv"

In [ ]:


# Import the Pinecone library
from pinecone import Pinecone as PineconeClient
from langchain.vectorstores import Pinecone as LangchainPinecone


os.environ["PINECONE_API_KEY"] = "pcsk_3gLL6_S9SetKF1d8DEnzXA6TdACPwjx5CuntHRY3BDH9vbJ652vVhtfXBq6RpgUfz1ijv"

# Initialize a Pinecone client with your API key
pc = PineconeClient(api_key=os.environ["PINECONE_API_KEY"])

# Create a dense index with integrated embedding
index_name = "med-bot"
if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"sentence-transformers/all-MiniLM-L6-v2",
            "field_map":{"text": "chunk_text"}
        }
    )


In [68]:
from langchain_pinecone import PineconeVectorStore

index = pc.Index(index_name)
dosearch_index = PineconeVectorStore.from_texts(texts=[t.page_content for t in text_chunks],
    embedding=embeddings,
    index_name=index_name)

In [69]:
#if we aleardy have an index, we can use it directly
dosearch_index = PineconeVectorStore.from_existing_index(
index_name=index_name,
embedding=embeddings,
)

query = "What is the treatment for diabetes?"
doc = dosearch_index.similarity_search(query, k=3)
print("Query Result: ", doc)

Query Result:  [Document(id='103e9345-48fa-458e-b4a1-cad7e6b41bdd', metadata={}, page_content='Treatment\nTreatment depends on the extent of disease and\nthe present health status of the patient. Additionally,\nin some cases treatment may stopped at sometime, or\nit may altogether be refused. Treatment for cardio-\nvascular disease may include surgical intervention\nand/or conservative medical treatment with medica-\ntions. Diet,exercise, and weight reduction are impor-\ntant parameters for treatment planning. Appropriate\nreferrals, counseling, and follow up are usually indi-\ncated. Treatment for cancer may include a combina-\ntion of surgery, chemotherapy or radiation therapy.\nThese treatment modalities may be given singly or in\ncombination or at different times during disease pro-\ngression. Cerebrovascular disease can be treated sur-\ngically and/or with medications that thin the blood.\nSymptomatic care may be indicated in addition to\nclose monitoring if the patient develops d

In [70]:
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: {context}
Question: {question}
Only return the answer, do not return any other text.
Helpful Answer:
"""

In [71]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

In [74]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                   model_type="llama",
                   config={"temperature": 0.8, "max_new_tokens": 512})

In [ ]:
qa = RetrievalQA.from_chain_type(
    chain_type="stuff",
    llm=llm,
    retriever=dosearch_index.as_retriever(search_kwargs={"k": 2}),
    chain_type_kwargs=chain_type_kwargs,
    return_source_documents=True
)

In [ ]:
while True:
    user_input = input("Input Prompt: ")
    result = qa.invoke({"query": user_input})
    print("Result:", result['result'])

C:\Users\QC\AppData\Local\Temp\ipykernel_1516\2053143468.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=qa({"query": user_input})
Number of tokens (675) exceeded maximum context length (512).
